In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from shapely.geometry import LineString

import geopandas as gpd
import geopy

tqdm.pandas()
sns.set(style="whitegrid")
import warnings 
warnings.filterwarnings("ignore")

In [2]:
incidents = pd.read_csv('DATA.csv')

In [3]:
incidents.head(1)

,Folder name,Folder color,Latitude,Longitude,Title,Description,Color,Phone number,Timestamp,Pin icon code
0,NaN,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,NaN,2024-02-08T17:16:00Z,-1


In [4]:
incidents.tail(1)

,Folder name,Folder color,Latitude,Longitude,Title,Description,Color,Phone number,Timestamp,Pin icon code
122,NaN,71b300,37.089224,-8.298158,1 de Setembro de 1986,Bombas Detonadas em Empreendimentos Turísitico...,3f51b5,NaN,2024-02-08T17:57:28Z,-1


In [5]:
incidents['ID'] = range(1, len(incidents) + 1)
incidents.set_index('ID', inplace = True)

In [6]:
incidents

,Folder name,Folder color,Latitude,Longitude,Title,Description,Color,Phone number,Timestamp,Pin icon code
ID,,,,,,,,,,
1,NaN,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,NaN,2024-02-08T17:16:00Z,-1
2,NaN,71b300,38.010202,-7.870127,1 de Fevereiro de 1985,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5,NaN,2024-02-08T17:37:17Z,-1
3,NaN,71b300,38.748041,-9.188370,1 de Junho de 1984,"Agressão, disparos contra administrador de uma...",ff5722,NaN,2024-02-08T17:10:19Z,-1
4,NaN,71b300,39.973942,-8.627772,1 de Maio de 1984,"Vandalismo, colocação de Pregos na Estrada Nac...",ffeb3b,NaN,2024-02-08T17:06:30Z,-1
5,NaN,71b300,38.815583,-9.223445,2 de Novembro de 1983,Assalto à dependência do Crédito Predial Portu...,71b300,NaN,2024-02-08T14:42:44Z,-1
...,...,...,...,...,...,...,...,...,...,...
119,NaN,71b300,38.768374,-9.294940,1 de Janeiro de 1982,Bombas Detonadas no Posto da Guarda Nacional R...,3f51b5,NaN,2024-02-08T13:58:46Z,-1
120,NaN,71b300,39.694550,-8.130265,1 de Julho de 1980,Constante Vandalismo contra veiculos da Políci...,ffeb3b,NaN,2024-02-08T12:23:55Z,-1
121,NaN,71b300,38.697433,-9.423230,1 de Maio de 1981,Explosivos Detonados no Royal British Club em ...,3f51b5,NaN,2024-02-08T13:28:17Z,-1


In [7]:
columns_to_drop = ['Folder name', 'Phone number', 'Pin icon code', 'Timestamp']
incidents.drop(columns=columns_to_drop, inplace = True)

In [8]:
incidents.head(1)

,Folder color,Latitude,Longitude,Title,Description,Color
ID,,,,,,
1,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300


In [9]:
incidents.isnull().sum()

Folder color    0
Latitude        0
Longitude       0
Title           0
Description     0
Color           0
dtype: int64

In [10]:
incidents['Color'].unique()

array(['71b300', '3f51b5', 'ff5722', 'ffeb3b', 'f44336', 'ff9800',
       '9e9e9e', '795548', '9c27b0', '607d8b'], dtype=object)

In [11]:
incidents['Folder color'].unique()

array(['71b300'], dtype=object)

In [12]:
incidents.drop('Folder color', inplace= True, axis = 1)

In [13]:
incidents.head(3)

,Latitude,Longitude,Title,Description,Color
ID,,,,,
1,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300
2,38.010202,-7.870127,1 de Fevereiro de 1985,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5
3,38.748041,-9.188370,1 de Junho de 1984,"Agressão, disparos contra administrador de uma...",ff5722


In [14]:
incidents['Color'].replace('ff9800', 'ff5722', inplace=True)

In [15]:
incident_type = {'71b300' : 'Assalto', 
                '3f51b5': 'Bomba Detonadas',
                'ff5722' : 'Agressão',
                'ffeb3b' : 'Vandalismo',
                'f44336' : 'Assassinato',
                '9e9e9e' : 'Assalto com Vítimas Mortais', 
                '795548' : 'Tentativa de Assalto (falhada)', 
                '9c27b0' : 'Bombas Não Detonadas',
                '607d8b' : 'Fugas da Prisão'
                 }

incidents['Categoria'] = incidents['Color'].map(incident_type)

In [16]:
incidents

,Latitude,Longitude,Title,Description,Color,Categoria
ID,,,,,,
1,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,Assalto
2,38.010202,-7.870127,1 de Fevereiro de 1985,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5,Bomba Detonadas
3,38.748041,-9.188370,1 de Junho de 1984,"Agressão, disparos contra administrador de uma...",ff5722,Agressão
4,39.973942,-8.627772,1 de Maio de 1984,"Vandalismo, colocação de Pregos na Estrada Nac...",ffeb3b,Vandalismo
5,38.815583,-9.223445,2 de Novembro de 1983,Assalto à dependência do Crédito Predial Portu...,71b300,Assalto
...,...,...,...,...,...,...
119,38.768374,-9.294940,1 de Janeiro de 1982,Bombas Detonadas no Posto da Guarda Nacional R...,3f51b5,Bomba Detonadas
120,39.694550,-8.130265,1 de Julho de 1980,Constante Vandalismo contra veiculos da Políci...,ffeb3b,Vandalismo
121,38.697433,-9.423230,1 de Maio de 1981,Explosivos Detonados no Royal British Club em ...,3f51b5,Bomba Detonadas


In [17]:
incidents.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 123 entries, 1 to 123
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Latitude     123 non-null    float64
 1   Longitude    123 non-null    float64
 2   Title        123 non-null    object 
 3   Description  123 non-null    object 
 4   Color        123 non-null    object 
 5   Categoria    123 non-null    object 
dtypes: float64(2), object(4)
memory usage: 48.3 KB


In [18]:
incidents = incidents.astype({'Color': 'category',  
                          'Categoria': 'category'})


In [19]:
incidents.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 123 entries, 1 to 123
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Latitude     123 non-null    float64 
 1   Longitude    123 non-null    float64 
 2   Title        123 non-null    object  
 3   Description  123 non-null    object  
 4   Color        123 non-null    category
 5   Categoria    123 non-null    category
dtypes: category(2), float64(2), object(2)
memory usage: 34.0 KB


In [20]:
incidents

,Latitude,Longitude,Title,Description,Color,Categoria
ID,,,,,,
1,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,Assalto
2,38.010202,-7.870127,1 de Fevereiro de 1985,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5,Bomba Detonadas
3,38.748041,-9.188370,1 de Junho de 1984,"Agressão, disparos contra administrador de uma...",ff5722,Agressão
4,39.973942,-8.627772,1 de Maio de 1984,"Vandalismo, colocação de Pregos na Estrada Nac...",ffeb3b,Vandalismo
5,38.815583,-9.223445,2 de Novembro de 1983,Assalto à dependência do Crédito Predial Portu...,71b300,Assalto
...,...,...,...,...,...,...
119,38.768374,-9.294940,1 de Janeiro de 1982,Bombas Detonadas no Posto da Guarda Nacional R...,3f51b5,Bomba Detonadas
120,39.694550,-8.130265,1 de Julho de 1980,Constante Vandalismo contra veiculos da Políci...,ffeb3b,Vandalismo
121,38.697433,-9.423230,1 de Maio de 1981,Explosivos Detonados no Royal British Club em ...,3f51b5,Bomba Detonadas


In [21]:
import re
from datetime import datetime

In [22]:
month_mapping = {
    'Janeiro': 1,
    'Fevereiro': 2,
    'Março': 3,
    'Abril': 4,
    'Maio': 5,
    'Junho': 6,
    'Julho': 7,
    'Agosto': 8,
    'Setembro': 9,
    'Outubro': 10,
    'Novembro': 11,
    'Dezembro': 12
}

def convert_title_to_date(title):

    match = re.search(r'(\d+) de (\w+) de (\d+)', title)
    if match:
        day, month_text, year = match.groups()
        month = month_mapping.get(month_text, 0)
        if month != 0:
            date_str = f"{year}-{month:02d}-{day}"
            return pd.to_datetime(date_str, format="%Y-%m-%d")
    return None


incidents['Data'] = incidents['Title'].apply(convert_title_to_date)
incidents.drop(columns=['Title'], inplace=True)


incidents.head()

,Latitude,Longitude,Description,Color,Categoria,Data
ID,,,,,,
1,41.336001,-8.560611,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,Assalto,1984-08-01
2,38.010202,-7.870127,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5,Bomba Detonadas,1985-02-01
3,38.748041,-9.188370,"Agressão, disparos contra administrador de uma...",ff5722,Agressão,1984-06-01
4,39.973942,-8.627772,"Vandalismo, colocação de Pregos na Estrada Nac...",ffeb3b,Vandalismo,1984-05-01
5,38.815583,-9.223445,Assalto à dependência do Crédito Predial Portu...,71b300,Assalto,1983-11-02


In [23]:
incidents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123 entries, 1 to 123
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Latitude     123 non-null    float64       
 1   Longitude    123 non-null    float64       
 2   Description  123 non-null    object        
 3   Color        123 non-null    category      
 4   Categoria    123 non-null    category      
 5   Data         120 non-null    datetime64[ns]
dtypes: category(2), datetime64[ns](1), float64(2), object(1)
memory usage: 5.8+ KB


In [28]:
import plotly.express as px

# Criar um mapa interativo usando Plotly Express focado em Portugal
map_fig = px.scatter_mapbox(incidents,
                            lat='Latitude',
                            lon='Longitude',
                            hover_name='Data',  # Mostrar a data quando o mouse passar sobre um ponto
                            color='Categoria',  # Diferenciar pontos pela categoria do incidente
                            size_max=15,  # Ajustar o tamanho máximo dos marcadores
                            zoom=5.5,  # Ajustando o nível de zoom para focar em Portugal
                            height=600,
                            width=800,
                            center={"lat": 39.5, "lon": -8.0},  # Coordenadas centrais de Portugal
                            mapbox_style="open-street-map")  # Usar estilo que não requer chave API

# Ajustando a aparência dos marcadores
map_fig.update_traces(marker=dict(size=15,  # Tamanho dos marcadores
                                  ))  # Cor dos marcadores para melhor visibilidade

# Mostrar o mapa
map_fig.show()
